# `reuters_mlp`
Trains and evaluate a simple multilayer perceptron (MLP) on the Reuters newswire topic classification task.

Can achieve accuracy around 80% after 5 epochs in ca. 25s on CPU.

### Import packages

In [1]:
from __future__ import print_function
import numpy as np

from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer

Using Theano backend.


### Set parameters
First, we specify a seed for the random number generator to obtain reproducible results.

In [2]:
np.random.seed(1337)

Now we specify the model and training parameters.

In [3]:
max_words = 1000
batch_size = 32
nb_epoch = 5

### Import and prepare data
We use the Reuters dataset in this example. It will be downloaded from the Internet during the first run. Subsequently, a cached version will be used.

80% of the database is used for training and 20% for prediction testing.

In [4]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(nb_words=max_words, test_split=0.2)

Let's see what we have...

In [5]:
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

nb_classes = np.max(y_train)+1
print(nb_classes, 'classes')

8982 train sequences
2246 test sequences
46 classes


We now vectorize sequence data:

In [6]:
tokenizer = Tokenizer(nb_words=max_words)
X_train = tokenizer.sequences_to_matrix(X_train, mode='binary')
X_test = tokenizer.sequences_to_matrix(X_test, mode='binary')
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (8982, 1000)
X_test shape: (2246, 1000)


And we have to convert class vector to a binary class matrix
so that it can be used with the `categorical_crossentropy` loss function.

In [7]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
print('Y_train shape:', Y_train.shape)
print('Y_test shape:', Y_test.shape)

Y_train shape: (8982, 46)
Y_test shape: (2246, 46)


### Model specification
We start off with creating a `Sequential` model for our MLP:

In [8]:
model = Sequential()

The first layer is a `Dense` layer with 512 outputs followed by a rectified linear unit (ReLU) activation layer.

In [9]:
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))

The second layer is a `Dropout` regularization layer, which prevents overfitting during training. We set the dropout ratio to 50%.

In [10]:
model.add(Dropout(0.5))

The output layer is a `Dense` layer with softmax activation.

In [11]:
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

Now we compile the model. We use categorical cross-entropy as a loss function and Adam as an optimizer.

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Model fitting
Now we fit the model!

In [13]:
history = model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1, validation_split=0.1);

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 5s - loss: 1.4274 - acc: 0.6812 - val_loss: 1.0934 - val_acc: 0.7553
Epoch 2/5
8083/8083 [==============================] - 5s - loss: 0.7737 - acc: 0.8169 - val_loss: 0.9172 - val_acc: 0.7920
Epoch 3/5
8083/8083 [==============================] - 5s - loss: 0.5441 - acc: 0.8691 - val_loss: 0.8529 - val_acc: 0.8087
Epoch 4/5
8083/8083 [==============================] - 4s - loss: 0.4132 - acc: 0.8968 - val_loss: 0.8730 - val_acc: 0.8076
Epoch 5/5
8083/8083 [==============================] - 5s - loss: 0.3338 - acc: 0.9180 - val_loss: 0.8931 - val_acc: 0.8176


### Model evalutaion
We can easily evaluate the trained model using the test set:

In [14]:
score = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=1)

2240/2246 [============================>.] - ETA: 0s

In [15]:
print('Test score:    {:.4f}'.format(score[0]))
print('Test accuracy: {:.4f}'.format(score[1]))

Test score:    0.8698
Test accuracy: 0.7952
